In [62]:
# R and Q are the covariance of the motion and observation models
R = 0.2
Q = 0.2
graphics_radius = 0.1

x_est = np.array([0, 1.5, 2.4, 3.4])
obs = np.array([ 2.9,  2.0, 1.0, 0.0 ])
x_true = np.array([0, 1, 2, 3])
landmark = 3
assert len(x_est) == len(obs) == len(x_true) 
N = len(x_est)

# Omega is the information matrix, which is used in negative_log_likelihood = error^T * Omega * error 
# So to minimize the max likelihood of the pose trajectory, we want to minimize negative_log_likelihood
# for likelihood, we assume the error is Gaussian: e*e/sigma^2, so to reflect that, Omega = [1/sigma^2] 
# in this case
omega = 1/Q

# counting combinations is enough for calculating the cost between all combos
node_pairs = list(itertools.combinations(range(N), 2))
for _ in range(4):
    # e_ij = z_j_hat - z_i_hat, where z_i_hat = x_i + obs_i
    H = np.zeros((N, N))
    b = np.zeros((N, 1))
    for i, j in node_pairs:
        # error is the difference between world frame observations from two nodes
        e_ij = x_est[j] + obs[j] - (x_est[i] + obs[i])
        # Jacobian of error for node i and j, with respect to all changes in pose estimates:
        # j_ij = [..., -1, ... 1, ...], 
        # b_ij = e_ij ^ T [Q] J_ij 
        J_ij = np.zeros((N,1))
        J_ij[i] = -1
        J_ij[j] = 1
        b_ij = e_ij * omega * J_ij
        b += b_ij
        
        H_ij = np.zeros((N, N))
        # so H = J^T Q J => H_ii = Q = H_jj; H_ij = -Q; H_ji = -Q
        H_ij[i, i] = H_ij[j, j] = omega
        H_ij[i, j] = H_ij[j, i] = -omega
        H += H_ij
    # Important: adding H[0,0] = 100 to assert 
    # "any delta x from node 0 to itself will be amplified large, because we are confident about node 0"
    H[0,0] += 100
    #TODO Remember to remove
    print(f'Rico: ===============')
    print(f'H : {H}')
    print(f'b: {b}')
    # Note, it's dx = H^-1 * -b
    dx = np.linalg.inv(H) @ (-b)
    x_est += dx.flatten()
    print(f'x_est: {x_est}')



Rico: ===============
H : [[115.  -5.  -5.  -5.]
 [ -5.  15.  -5.  -5.]
 [ -5.  -5.  15.  -5.]
 [ -5.  -5.  -5.  15.]]
b: [[-8.]
 [ 4.]
 [ 2.]
 [ 2.]]
x_est: [0.  0.9 1.9 2.9]
Rico: ===============
H : [[115.  -5.  -5.  -5.]
 [ -5.  15.  -5.  -5.]
 [ -5.  -5.  15.  -5.]
 [ -5.  -5.  -5.  15.]]
b: [[0.]
 [0.]
 [0.]
 [0.]]
x_est: [0.  0.9 1.9 2.9]
Rico: ===============
H : [[115.  -5.  -5.  -5.]
 [ -5.  15.  -5.  -5.]
 [ -5.  -5.  15.  -5.]
 [ -5.  -5.  -5.  15.]]
b: [[0.]
 [0.]
 [0.]
 [0.]]
x_est: [0.  0.9 1.9 2.9]
Rico: ===============
H : [[115.  -5.  -5.  -5.]
 [ -5.  15.  -5.  -5.]
 [ -5.  -5.  15.  -5.]
 [ -5.  -5.  -5.  15.]]
b: [[0.]
 [0.]
 [0.]
 [0.]]
x_est: [0.  0.9 1.9 2.9]
